### Introduction: This is the Jupyter notebook to do the flowing things:

1. Read slimmed PKU Tree files
2. Store the raw MC histograms to pickle files

kernel:HWW



### Import necessary modules

In [1]:
import numpy as np
import os
import pandas as pd
import random
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from scipy import interpolate
# from sklearn.metrics import roc_curve, auc
from cycler import cycler
import uproot
# means uproot4
import ROOT
import argparse
import glob
import json
import logging
import os
import pickle as pkl
import warnings

import hist as hist2
import pyarrow
# import utils #local file: utils.py
import yaml
from typing import Dict, List, Union
from dataclasses import dataclass, field
from copy import deepcopy

# from coffea import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods import vector
# from coffea.nanoevents.methods.vector import PtEtaPhiMLorentzVector

Welcome to JupyROOT 6.28/04


In [2]:
YEAR = "2018"

### Read SlimmedTree files

In [3]:
#load the slimmedtree files using uproot

#different year available here.
# year = "2016APV"
# year = "2016"
# year = "2017"
# year = "2018"
# year = "Full-Run2"
year = YEAR

#if run on PKU cluster, use this:
# CustNanoData = {
#     'data'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Data/SlimmedTree_Data.root"%(year),
#     'QCD'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_QCD.root"%(year),
#     'Top'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_Top.root"%(year),
#     'WJets'       : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_WJets.root"%(year),
#     'Rest'        : "/data/bond/zhaoyz/SlimmedTree/V5/%s/MC/SlimmedTree_Rest.root"%(year),
#     'TotalSignal' : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_Total.root"%(year),
#     'ggF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_GluGlu.root"%(year),
#     'VH'          : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VH.root"%(year),
#     'ttH'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_ttH.root"%(year),
#     'VBF'         : "/data/bond/zhaoyz/SlimmedTree/V5/%s/Signal/SlimmedTree_VBF.root"%(year),
# } 

#if run on lxplus, use this:
# CustNanoData = {
#     'data'        : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Data/SlimmedTree_Data.root"%(year),
#     'QCD'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_QCD.root"%(year),
#     'Top'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_Top.root"%(year),
#     'WJets'       : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_WJets.root"%(year),
#     'Rest'        : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/MC/SlimmedTree_Rest.root"%(year),
#     'TotalSignal' : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_Total.root"%(year),
#     'ggF'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_GluGlu.root"%(year),
#     'VH'          : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_VH.root"%(year),
#     'ttH'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_ttH.root"%(year),
#     'VBF'         : "/eos/user/y/yuzhe/HWW/SlimmedTree/V5/%s/Signal/SlimmedTree_VBF.root"%(year),
# }  

#if run on CMSconnect, use this:
CustNanoData = {
    'data'        : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Data/SlimmedTree_Data.root"%(year),
    'QCD'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_QCD.root"%(year),
    'TT'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_TT.root"%(year),
    'ST'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_ST.root"%(year),
    'WJets'       : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_WJets.root"%(year),
    'Rest'        : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/MC/SlimmedTree_Rest.root"%(year),
    'TotalSignal' : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_Total.root"%(year),
    'ggF'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_GluGlu.root"%(year),
    'ZH'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_WH.root"%(year),
    'WH'          : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_HZJ.root"%(year),
    'ttH'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_ttH.root"%(year),
    'VBF'         : "/ospool/cms-user/yuzhe/SlimmedTree/V6/%s/Signal/SlimmedTree_VBF.root"%(year),
}        

files = {typefile : {} for typefile in CustNanoData}
for typefile in CustNanoData:
    files[typefile] = uproot.lazy({CustNanoData[typefile]: "PKUTree" })
    
#for signal decomposition plots, we store these information in different objects
files[r"$H^{4q}$"] = files["TotalSignal"][files["TotalSignal"]["R4q_a"] == 1]
files[r"$H^{3q}$"] = files["TotalSignal"][(files["TotalSignal"]["R3q_a"] == 1) & (files["TotalSignal"]["R3q_taudecay_a"] != 1)]
files["W"] = files["TotalSignal"][files["TotalSignal"]["w_a"] == 1]
files["top"] = files["TotalSignal"][files["TotalSignal"]["t_a"] == 1]
files["Z"] = files["TotalSignal"][files["TotalSignal"]["z_a"] == 1]
files[r"$H^{lqq}$"] = files["TotalSignal"][(files["TotalSignal"]["Rlqq_a"] == 1) | (files["TotalSignal"]["R3q_taudecay_a"] == 1)]
files["g/q"] = files["TotalSignal"][files["TotalSignal"]["gKK_g_a"] == 1]
files["rest"] = files["TotalSignal"][(files["TotalSignal"]["u_a"] == 1) | (files["TotalSignal"]["Rlq_a"] == 1) | (files["TotalSignal"]["R2q_a"] == 1)]


### Get DPhi in the events

In [4]:
def get_dphi(events):
    pT_higgs   = events["PTj_V2_a"]
    eta_higgs  = events["Etaj_V2_a"]
    phi_higgs  = events["Phij_V2_a"]
    mass_higgs = events["Mj_V2_a"]
    pT_MET = events["MET_et"]
    eta_MET = events["Etaj_V2_a"]
    phi_MET = events["MET_phi"]
    mass_MET = ak.zeros_like(events["MET_phi"])
    vec_higgs = ak.zip({
        "pt"   : pT_higgs   ,
        "eta"  : eta_higgs  ,
        "phi"  : phi_higgs  ,
        "mass" : mass_higgs ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )
    vec_MET = ak.zip({
        "pt"   :   pT_MET   ,
        "eta"  :  eta_MET  ,
        "phi"  :  phi_MET  ,
        "mass" : mass_MET ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )
    delta_phi = np.subtract(vec_MET.phi, vec_higgs.phi)
    delta_phi = np.where(delta_phi > np.pi, delta_phi - 2*np.pi, delta_phi)
    delta_phi = np.where(delta_phi < -np.pi, delta_phi + 2*np.pi, delta_phi)
    delta_phi = np.abs(delta_phi)
    print(delta_phi)
    events["DPhi"] = delta_phi

In [5]:
for k in files:
    print("Add dphi of:",k)
    get_dphi(events=files[k])

Add dphi of: data
[0.719, 0.16, 2.82, 0.0338, 2.16, 2.61, ... 0.767, 0.2, 0.504, 3.03, 0.289, 0.0353]
Add dphi of: QCD
[3.07, 0.27, 2.35, 2.65, 3.1, 3.14, 2.42, ... 0.0396, 3.07, 0.765, 3, 2.96, 3.07]
Add dphi of: TT
[0.0237, 1.14, 3.12, 2.69, 2.79, 0.804, ... 2.96, 3.07, 0.281, 2.05, 0.863, 0.608]
Add dphi of: ST
[0.876, 3.04, 1.45, 0.137, 3.13, 2.7, 0.281, ... 2.5, 0.137, 3.12, 1.26, 0.222, 1.97]
Add dphi of: WJets
[2.57, 0.195, 2.46, 2.95, 0.0653, 2.37, ... 0.185, 2.52, 2.93, 0.253, 2.14, 3.02]
Add dphi of: Rest
[2.97, 3.08, 0.344, 0.158, 1.35, 0.194, ... 1.26, 1.24, 0.0949, 2.46, 1.87, 2.91]
Add dphi of: TotalSignal
[0.486, 0.0852, 2.85, 0.551, 1.49, 0.535, ... 0.743, 0.839, 2.31, 2.44, 2.99, 0.506]
Add dphi of: ggF
[0.486, 0.0852, 2.85, 0.551, 1.49, 0.535, ... 3.09, 0.722, 3, 3.11, 3.07, 0.455]
Add dphi of: ZH
[3.03, 1.91, 0.382, 2.28, 0.302, 3.03, 0.289, ... 0.419, 1.5, 2.61, 0.299, 1.9, 1.2]
Add dphi of: WH
[2.26, 3.03, 0.15, 1.3, 0.796, 2.76, 3.12, ... 3.1, 0.141, 0.273, 0.498,

### Get MET recovery mass

In [6]:

def get_reco(events,mass = "Mj_corr_V2_a", MET_UE = None):
    pT_higgs   = events["PTj_V2_a"]
    eta_higgs  = events["Etaj_V2_a"]
    phi_higgs  = events["Phij_V2_a"]
    mass_higgs = events[mass]
    if not MET_UE:
        pT_MET = events["MET_et"]
        phi_MET = events["MET_phi"]
    elif MET_UE == "up":
        pT_MET = events["MET_et_UEup"]
        phi_MET = events["MET_phi_UEup"]
    elif MET_UE == "down":
        pT_MET = events["MET_et_UEdown"]
        phi_MET = events["MET_phi_UEdown"]
    eta_MET = events["Etaj_V2_a"]
    mass_MET = ak.zeros_like(events["MET_phi"])
    vec_higgs = ak.zip({
        "pt"   : pT_higgs   ,
        "eta"  : eta_higgs  ,
        "phi"  : phi_higgs  ,
        "mass" : mass_higgs ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )

    vec_MET = ak.zip({
        "pt"   :   pT_MET   ,
        "eta"  :  eta_MET  ,
        "phi"  :  phi_MET  ,
        "mass" : mass_MET ,
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
    )

    vec_sum = vec_MET + vec_higgs
    mH_reco = vec_sum.mass
    if not MET_UE: mass_str = "MH_Reco" if mass == "Mj_corr_V2_a" or mass == "Mj_V2_a" else mass
    else : mass_str = "MH_Reco" + "_UE_" + MET_UE
    
    if not MET_UE: 
        events[mass_str] = ak.where(( (events["DPhi"] < 0.8) & (events["MET_et"]/events["PTj_V2_a"] > 0.1)), mH_reco, events[mass])
    elif MET_UE == "up" : 
        events[mass_str] = ak.where(( (events["DPhi"] < 0.8) & (events["MET_et_UEup"]/events["PTj_V2_a"] > 0.1)), mH_reco, events[mass])
    elif MET_UE == "down" : 
        events[mass_str] = ak.where(( (events["DPhi"] < 0.8) & (events["MET_et_UEdown"]/events["PTj_V2_a"] > 0.1)), mH_reco, events[mass])

In [7]:
for k in files:
    # note that QCD and Data don't have such variation
    if k == "data" or k == "QCD": continue
    print("Add reco of:",k)
    get_reco(events=files[k])
    get_reco(events=files[k],MET_UE = "up")
    get_reco(events=files[k],MET_UE = "down")
    
    get_reco(events=files[k],mass = "Mj_jesTotalUp_a")
    get_reco(events=files[k],mass = "Mj_jesTotalDown_a")
    get_reco(events=files[k],mass = "Mj_jerUp_a")
    get_reco(events=files[k],mass = "Mj_jerDown_a")
    
    get_reco(events=files[k],mass = "Mj_jmsUp_a")
    get_reco(events=files[k],mass = "Mj_jmsDown_a")
    get_reco(events=files[k],mass = "Mj_jmrUp_a")
    get_reco(events=files[k],mass = "Mj_jmrDown_a")
for k in ["QCD","data"]:
    get_reco(events=files[k],mass = "Mj_V2_a")

Add reco of: TT
Add reco of: ST
Add reco of: WJets
Add reco of: Rest
Add reco of: TotalSignal
Add reco of: ggF
Add reco of: ZH
Add reco of: WH
Add reco of: ttH
Add reco of: VBF
Add reco of: $H^{4q}$
Add reco of: $H^{3q}$
Add reco of: W
Add reco of: top
Add reco of: Z
Add reco of: $H^{lqq}$
Add reco of: g/q
Add reco of: rest


### some test about variables / output all the variables

In [8]:
files["TotalSignal"].fields

['Mj_jmrDown_c',
 't_b',
 'Mj_min',
 'R2q_c',
 'Rlq_b',
 'PSWeight_1',
 'PTj_2',
 'Mj_jesTotalDown_c',
 'PTj',
 'a_HWqqWq1c',
 'nb_m_deep_ex',
 'a_HWqqWtaumv1c',
 'Phij_V2_b',
 'Mj_jerUp_b',
 'a_TopbWqq1c',
 'a_QCDcc',
 'a_Hcc',
 'Mj_corr',
 'LHEScaleWeight_0',
 'Phij_max',
 'MET_T1Smear_phi_XYcorr',
 'Nj8',
 'nb_t_deep_ex',
 'Phij',
 'Mj_jerDown_c',
 'a_HWqqWtauev0c',
 'Phij_3',
 'Nj4_ex',
 'PTj_3',
 'gKK_g_b',
 'z_b',
 'Phij_min',
 'gKK_g_c',
 'Mj_corr_V2_b',
 'a_TopbWtauhv',
 'MET_phi_UEup',
 'R3q_taudecay_c',
 'Phij_2',
 'Mj_corr_b',
 'a_HWWvsQCD_V2',
 'a_Hqq',
 'a_HWqqWmv0c',
 'Mj_corr_c',
 'nb_t_deep_in',
 'a_HWqqWtaumv0c',
 'a_TopbWtauev',
 'Rlqq_a',
 'Etaj_3',
 'Mj_V2_c',
 'R4q_b',
 'u_b',
 'LHEScaleWeight_2',
 'MET_et_UEdown',
 'Rlq_c',
 'c_HWW_V2',
 'a_HWqqWtauev1c',
 'Mj_jerUp_c',
 'a_HWqqWq2c',
 'a_TopbWq0c',
 'R3q_a',
 'R3q_c',
 'tlqq_c',
 'SF_unc',
 'Mj_jmrUp_b',
 'Rlq_a',
 'Mj_jmsDown_b',
 'nb_l_deep_in',
 'PTj_max',
 'Etaj_V2_a',
 'b_HWW_V2',
 'Mj_jesTotalUp_b',
 'LHESc

### plot setting

In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
from cycler import cycler

use_helvet = False ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet['font.sans-serif'] = ['Helvetica', 'Arial']
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

def flow(hist: bh.Histogram, overflow: bool=True, underflow: bool=True):
    h, var = hist.view(flow=(overflow | underflow)).value, hist.view(flow=(overflow | underflow)).variance
    if overflow: 
        # h, var also include underflow bins but in plots usually no underflow data
        # And we've filled None with -999, so we shouldn't show underflow data (mostly from filled None)
        # You have to access the overflow and underflow bins data like below:
        h[-2] += h[-1]; var[-2] += var[-1]
    if underflow:
        h[1] += h[0]; var[1] += var[0]
    if overflow or underflow:
        h, var = h[1:-1], var[1:-1]
    return h, var
    # Return the updated histogram and variance

def error_bar(h, var, type='data'):
    from scipy.interpolate import CubicSpline
    if type == 'data':
        number = h
    elif type == 'mc':  # h = k*N, var = k^2*N, std = k*sqrt(N)
        number = h**2 / var
    else:
        raise ValueError("type should be 'data' or 'mc'! ")
    center = range(11) # Number: 0-10
    up = np.array([1.84, 3.30, 4.64, 5.92, 7.16, 8.38, 9.58, 10.77, 11.95, 13.11, 14.27]) - center
    down = center - np.array([0, 0.17, 0.71, 1.37, 2.09, 2.84, 3.62, 4.42, 5.23, 6.06, 6.89])
    #cs means to create a CubicSpline object
    cs_up = CubicSpline(x=center, y=up)
    cs_down = CubicSpline(x=center, y=down)
    
    Garwood = (number>0)&(number<10)
    poison_error_bar = np.sqrt(number)
    up_error_bar = np.copy(poison_error_bar)
    down_error_bar = np.copy(poison_error_bar)
    up_error_bar[Garwood] = cs_up(number[Garwood])
    down_error_bar[Garwood] = cs_down(number[Garwood])
    if type == 'mc':
        up_error_bar *= var/h
        down_error_bar *= var/h
    up_error_bar [up_error_bar < 0 ] = 0
    down_error_bar [down_error_bar < 0 ] = 0
    return np.array([down_error_bar, up_error_bar])


# function to find the optimal region with S/sqrt(B)
# not used so far
def optimalcut(shist, bhist):
    n_bins = len(shist)
    best_lower = None
    best_upper = None
    best_s_sqrt_b = 0

    for lower in range(n_bins):
        for upper in range(lower+1, n_bins+1):
            s = np.sum(shist[lower:upper])
            b = np.sum(bhist[lower:upper])
            s_sqrt_b = s / np.sqrt(b + 1)

            if s_sqrt_b > best_s_sqrt_b:
                best_lower = lower
                best_upper = upper
                best_s_sqrt_b = s_sqrt_b

    return best_lower, best_upper, best_s_sqrt_b

def optimalcut_oneside(shist, bhist, epsilon = 0.01):
    '''
    Given the signal histogram and background histogram, 
    show the one-side cut for the variable to get best s/sqrt(b).
    Args:
        shist:signal histogram
        bhist:background histogram
        epsilon(float): epsilon to avoid numerical errs 
    '''
    n_bins = len(shist)
    best_cut = 0
    best_s_sqrt_b = 0

    for cut in range(n_bins):
        s = np.sum(shist[cut:])
        b = np.sum(bhist[cut:])
        s_sqrt_b = s / np.sqrt(b + epsilon)
        if s_sqrt_b > best_s_sqrt_b:
            best_cut = cut
            best_s_sqrt_b = s_sqrt_b
        
    return best_cut, best_s_sqrt_b

def optimalcut_mid_combine(shist1, shist2, bhist, epsilon = 1):
    '''
    Given the signal histogram and background histogram, 
    show the one-side cut for the variable to get best s/sqrt(b).
    Args:
        shist:signal histogram
        bhist:background histogram
        epsilon(float): epsilon to avoid numerical errs 
    '''
    n_bins = len(shist1)
    best_cut = 0
    best_combined_sig_two_side = 0

    for cut in range(n_bins):
        s_right_side = np.sum(shist2[cut:])
        b_right_side = np.sum(bhist[cut:])
        s_left_side = np.sum(shist1[:cut])
        b_left_side = np.sum(bhist[:cut])
        s_sqrt_b_right_side = s_right_side / np.sqrt(b_right_side + epsilon)
        s_sqrt_b_left_side = s_left_side / np.sqrt(b_left_side + epsilon)
        combined_sig_two_side = np.sqrt((s_sqrt_b_right_side)**2 + (s_sqrt_b_left_side)**2)
        if combined_sig_two_side > best_combined_sig_two_side:
            best_cut = cut
            best_combined_sig_two_side = combined_sig_two_side
        
    return best_cut, best_combined_sig_two_side


### Define templates dir

In [46]:
MAIN_DIR = "."

plot_dir = f"{MAIN_DIR}/templates/20Mar24"
_ = os.system(f"mkdir -p {plot_dir}")

### Define observable object variables

In [11]:
@dataclass
class ShapeVar:
    """Class to store attributes of a variable to make a histogram of.

    Args:
        var (str): variable name
        label (str): variable label
        bins (List[int]): bins
        reg (bool, optional): Use a regular axis or variable binning. Defaults to True.
        blind_window (List[int], optional): if blinding, set min and max values to set 0. Defaults to None.
        significance_dir (str, optional): if plotting significance, which direction to plot it in.
          See more in plotting.py:ratioHistPlot(). Options are ["left", "right", "bin"]. Defaults to "right".
    """

    var: str = None
    label: str = None
    bins: List[int] = None
    reg: bool = True #regular axis
    blind_window: List[int] = None
    significance_dir: str = "right"

    def __post_init__(self):
        # create axis used for histogramming
        if self.reg:
            self.axis = hist2.axis.Regular(*self.bins, name=self.var, label=self.label)
        else:
            self.axis = hist2.axis.Variable(self.bins, name=self.var, label=self.label)

@dataclass
class Syst:
    samples: list[str] = None
    years: list[str] = field(default_factory=lambda: years)
    label: str = None
    
def blindBins(h: hist2.Hist, blind_region: List, blind_sample: str = None, axis=0):
    """
    Blind (i.e. zero) bins in histogram ``h``.
    If ``blind_sample`` specified, only blind that sample, else blinds all.
    """
    if axis > 0:
        raise Exception("not implemented > 1D blinding yet")

    bins = h.axes[axis + 1].edges
    lv = int(np.searchsorted(bins, blind_region[0], "right"))
    rv = int(np.searchsorted(bins, blind_region[1], "left") + 1)

    if blind_sample is not None:
        data_key_index = np.where(np.array(list(h.axes[0])) == blind_sample)[0][0]
        h.view(flow=True)[data_key_index][lv:rv].value = 0
        h.view(flow=True)[data_key_index][lv:rv].variance = 0
    else:
        h.view(flow=True)[:, lv:rv].value = 0
        h.view(flow=True)[:, lv:rv].variance = 0       
shape_vars = [
    ShapeVar(
        "MH_Reco",
        r"Higgs candidate MET recovery mass [GeV]",
        [20, 50, 250],
        reg=True,
        blind_window=[90, 150],
    )
]


### Define samples we consider

In [12]:
sig_keys = [
    "ggF",
    "VBF",
    "ZH",
    "WH",
    "ttH",
]

bkg_keys = [
    "TT",
    "WJets",
    "ST",
    "Rest"
]

mc_keys = sig_keys + bkg_keys

### Define weight shift list

In [13]:
years = ["2016APV", "2016", "2017", "2018"]

weight_shifts = {
    "pileup": Syst(samples=mc_keys, label="Pileup"),
    "ISRPartonShower": Syst(samples=mc_keys, label="ISR Parton Shower"),
    "FSRPartonShower": Syst(samples=mc_keys, label="FSR Parton Shower"),
    "QCDscale": Syst(samples=bkg_keys, label="QCDScale"),
    "trigger" : Syst(samples=mc_keys, label="Trigger SF"),
}



### Re-organize weight information

In [31]:
samples = list(['data','QCD','TT','ST','WJets','Rest','ggF','WH','ZH','ttH','VBF']) #all samples we considered
years = ["2016APV", "2016", "2017", "2018"]
year_to_run = YEAR
for shift in ["down", "up"]:
    print("processing:",shift)
    for wshift, wsyst in weight_shifts.items():
        for wsample in wsyst.samples:
            print("processing:",wsample)
            if wsample in samples:
                if wshift == "pileup" :
                    print("processing:",wshift)
                    if shift == "up":
                        for year in years: 
                            if year == year_to_run : 
                                files[wsample][f"{wshift}_{shift}_{year}"] = files[wsample]["weight"] * (files[wsample]["puWeightUp"] / files[wsample]["puWeight"])
                            else : 
                                files[wsample][f"{wshift}_{shift}_{year}"] = files[wsample]["weight"]
                    if shift == "down":
                        for year in years: 
                            if year == year_to_run : 
                                files[wsample][f"{wshift}_{shift}_{year}"] = files[wsample]["weight"] * (files[wsample]["puWeightDown"] / files[wsample]["puWeight"])
                            else : 
                                files[wsample][f"{wshift}_{shift}_{year}"] = files[wsample]["weight"]
                if wshift == "ISRPartonShower" :
                    print("processing:",wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_0"]
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_2"] 
                if wshift == "FSRPartonShower" :
                    print("processing:",wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_1"]
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["PSWeight_3"] 
                if wshift == "QCDscale" :
                    print("processing:",wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["LHEScaleWeight_8"]
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * files[wsample]["LHEScaleWeight_0"] 
                if wshift == "trigger" :
                    print("processing:",wshift)
                    if shift == "up":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (1 + files[wsample]["SF_unc"])
                    if shift == "down":
                        files[wsample][f"{wshift}_{shift}"] = files[wsample]["weight"] * (1 - files[wsample]["SF_unc"])

processing: down
processing: ggF
processing: pileup
processing: VBF
processing: pileup
processing: ZH
processing: pileup
processing: WH
processing: pileup


processing: ttH
processing: pileup
processing: TT
processing: pileup
processing: WJets
processing: pileup
processing: ST
processing: pileup
processing: Rest
processing: pileup
processing: ggF
processing: ISRPartonShower
processing: VBF
processing: ISRPartonShower
processing: ZH
processing: ISRPartonShower
processing: WH
processing: ISRPartonShower
processing: ttH
processing: ISRPartonShower
processing: TT
processing: ISRPartonShower
processing: WJets
processing: ISRPartonShower
processing: ST
processing: ISRPartonShower
processing: Rest
processing: ISRPartonShower
processing: ggF
processing: FSRPartonShower
processing: VBF
processing: FSRPartonShower
processing: ZH
processing: FSRPartonShower
processing: WH
processing: FSRPartonShower
processing: ttH
processing: FSRPartonShower
processing: TT
processing: FSRPartonShower
processing: WJets
processing: FSRPartonShower
processing: ST
processing: FSRPartonShower
processing: Rest
processing: FSRPartonShower
processing: TT
processing: QCDscal

In [32]:
files["WJets"]["pileup_down_2016APV"]

<Array [0.118, 0.121, 0.125, ... 1.75, 1.48] type='7453073 * float64'>

In [22]:
files["WJets"]["MH_Reco"]

<Array [139, 94.2, 201, ... 163, 205, 109] type='7453073 * float32'>

### Define variation shift list

In [41]:
jecs = {
    "JES": "JES_jes",
    "JER": "JER",
}

uncluste = {
    "UE": "unclusteredEnergy",
}

jec_shifts = {}
for key in jecs:
    for shift in ["up", "down"]:
        if key == "JES": 
            if shift == "up"   : jec_shifts[f"{key}_{shift}"] = "Mj_jesTotalUp_a"
            if shift == "down" : jec_shifts[f"{key}_{shift}"] = "Mj_jesTotalDown_a"
        if key == "JER": 
            if shift == "up"   : jec_shifts[f"{key}_{shift}"] = "Mj_jerUp_a"
            if shift == "down" : jec_shifts[f"{key}_{shift}"] = "Mj_jerDown_a"

ue_shifts = {}
for key in uncluste:
    for shift in ["up", "down"]:
        if shift == "up"   : ue_shifts[f"{key}_{shift}"] = "MH_Reco_UE_up"
        if shift == "down" : ue_shifts[f"{key}_{shift}"] = "MH_Reco_UE_down"

In [42]:
jec_shifts

{'JES_up': 'Mj_jesTotalUp_a',
 'JES_down': 'Mj_jesTotalDown_a',
 'JER_up': 'Mj_jerUp_a',
 'JER_down': 'Mj_jerDown_a'}

In [43]:
ue_shifts

{'UE_up': 'MH_Reco_UE_up', 'UE_down': 'MH_Reco_UE_down'}

### Define CUT(aka. regions)

In [33]:
CUT = {        
    "SR1a" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.99) for k in files}, 
    "SR1b" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) for k in files},   
    "SR2a" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.99) & (files[k]["DPhi"] < 0.8)for k in files}, 
    "SR2b" : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) & (files[k]["DPhi"] < 0.8)for k in files},   
    "CR1"  : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] < 0.92) for k in files},
    "CR2"  : {k:  (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25)  & (files[k]["a_HWW_V2"] < 0.92) & (files[k]["DPhi"] < 0.8) for k in files},   
    }

# CR: 0.6-0.8

In [34]:
CUT_BLINDED = {
        "SR1a_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.99) for k in files}, 
        "SR1b_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) for k in files},   
        "SR2a_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.99) & (files[k]["DPhi"] < 0.8)for k in files}, 
        "SR2b_blinded" : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25) & (files[k]["a_HWW_V2"] >= 0.92) & (files[k]["a_HWW_V2"] < 0.99) & (files[k]["DPhi"] < 0.8)for k in files},   
        "CR1_blinded"  : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] <= 0.25) & (files[k]["a_HWW_V2"] < 0.92) for k in files},
        "CR2_blinded"  : {k: (files[k]["nb_t_deep_in"] == 0 ) & (files[k]["MH_Reco"] >= 50) & ((files[k]["MH_Reco"] <= 90) | (files[k]["MH_Reco"] >= 150)) & (files[k]["MET_et"]/files[k]["PTj_V2_a"] > 0.25)  & (files[k]["a_HWW_V2"] < 0.92) & (files[k]["DPhi"] < 0.8)for k in files},   
}

### Save hist templates to pkl files

We note here that no particular operation is needed for QCD, since we only need raw QCD MC ratio as initial tranfer factor in the actual QCD prediction, and rhalphabet method will use (data - other bkg) in fail(control) region and perform simultaneous fit with pass(signal) region

In [49]:

def save_pkl(files, path_str = plot_dir, template_file = "templates",year_to_run = "2018"):
    
    templates = {} #empty dict to store the templates file
    regions = ["SR1a","SR1b","CR1","SR2a","SR2b","CR2"] #signal regions or control regions
    samples = list(['data','QCD','TT','ST','WJets','Rest','ggF','WH','ZH','ttH','VBF']) #all samples we considered
    
    #fill templates for different regions
    for region in regions:
        templates[region] = hist2.Hist(
            hist2.axis.StrCategory(samples, name="Sample"),
            *[shape_var.axis for shape_var in shape_vars],
            storage="weight",
            ) #initialize a hist object
        
        #add center value templates first
        for sample in samples:
            data = files[sample][CUT[region][sample]] if ("blinded" not in region) else files[sample][CUT_BLINDED[region][sample]]
            templates[region].fill(
                                Sample=sample,
                                MH_Reco=data["MH_Reco"],
                                weight=data["weight"],
                            )
            if sample == "data": 
                if (region.endswith("a") or region.endswith("b")):
                    # blind signal mass windows in pass region in data
                    print("blind data of ",region)
                    for i, shape_var in enumerate(shape_vars):
                        if shape_var.blind_window is not None:
                            blindBins(templates[region], shape_var.blind_window, "data", axis=i) 

        #add weight variation for each sample            
        for shift in ["down", "up"]:
            for wshift, wsyst in weight_shifts.items():
                if wshift == "pileup":
                    for year in years: 
                        # pileup need year specific information
                        for wsample in wsyst.samples:
                            if wsample in samples:
                                data = files[wsample][CUT[region][wsample]] if ("blinded" not in region) else files[wsample][CUT_BLINDED[region][wsample]]
                                print(region, wsample, wshift, shift)
                                print("mass info:",data["MH_Reco"])
                                print("weight infor:",data[f"{wshift}_{shift}_{year}"])
                                templates[region].fill(
                                    Sample=wsample + f"_{wshift}_{shift}",
                                    MH_Reco=data["MH_Reco"],
                                    weight=data[f"{wshift}_{shift}_{year}"],
                                )                    
                else:
                    for wsample in wsyst.samples:
                        if wsample in samples:
                            data = files[wsample][CUT[region][wsample]] if ("blinded" not in region) else files[wsample][CUT_BLINDED[region][wsample]]
                            templates[region].fill(
                                Sample=wsample + f"_{wshift}_{shift}",
                                MH_Reco=data["MH_Reco"],
                                weight=data[f"{wshift}_{shift}"],
                            )
                        
        #add shift variation for each sample
        for sample in mc_keys:
            #JECS
            for wshift, wsyst in jec_shifts.items():
                for year in years:
                    if year == year_to_run:
                        data = files[sample][CUT[region][sample]] if ("blinded" not in region) else files[sample][CUT_BLINDED[region][sample]]
                        templates[region].fill(
                                Sample=sample + f"_{wshift}_{year}",
                                MH_Reco=data[wsyst],
                                weight=data["weight"],
                            )
                    else:
                        data = files[sample][CUT[region][sample]] if ("blinded" not in region) else files[sample][CUT_BLINDED[region][sample]]
                        templates[region].fill(
                                Sample=sample + f"_{wshift}_{year}",
                                MH_Reco=data["MH_Reco"],
                                weight=data["weight"],
                            )                                    
            #un-clustered energy
            for wshift, wsyst in ue_shifts.items():
                data = files[sample][CUT[region][sample]] if ("blinded" not in region) else files[sample][CUT_BLINDED[region][sample]]
                templates[region].fill(
                                Sample=sample + f"_{wshift}",
                                MH_Reco=data[wsyst],
                                weight=data["weight"],
                            )               
            
        print("done fill template ",region)        
    
    #Creates blinded copies of each region's templates and saves a pickle of the templates
    blind_window = shape_vars[0].blind_window
    for label, template in list(templates.items()):
        blinded_template = deepcopy(template)
        blindBins(blinded_template, blind_window)
        templates[f"{label}Blinded"] = blinded_template
    #save files
    print(year_to_run)
    with open(f"{path_str}/{template_file}_{year_to_run}.pkl", "wb") as fp:
        pkl.dump(templates, fp) # dump the templates of each region in a pkl file
        print("Saved templates to ", f"{template_file}_{year_to_run}")

In [51]:
save_pkl(files = files, year_to_run = YEAR)


blind data of  SR1a
SR1a ggF pileup down
mass info: [146, 113, 128, 157, 153, 133, 124, 110, ... 76.3, 143, 88.7, 125, 154, 134, 137]
weight infor: [0.00922, 0.0141, 0.0103, 0.00853, 0.011, ... 0.00953, 0.0112, 0.0135, 0.0111]
SR1a VBF pileup down
mass info: [145, 132, 133, 91.9, 127, 161, 155, 92, ... 166, 118, 115, 79, 161, 141, 152, 82.4]
weight infor: [0.0139, 0.0186, 0.0146, 0.0152, 0.0147, ... 0.0164, 0.0165, 0.0167, 0.0129, 0.0143]
SR1a ZH pileup down
mass info: [164, 131, 149, 138, 120, 80.5, 121, 84.9, ... 133, 56.2, 120, 149, 131, 140, 129]
weight infor: [0.000586, 0.000599, 0.000554, 0.000709, ... 0.0011, 0.0012, 0.000967, 0.000983]
SR1a WH pileup down
mass info: [162, 157, 125, 183, 112, 115, 83.9, 134, ... 100, 139, 129, 141, 126, 261, 123, 130]
weight infor: [0.00126, -0.00107, 0.00153, 0.00126, ... 0.00128, 0.00122, 0.000918, 0.00106]
SR1a ttH pileup down
mass info: [128, 58.9, 96.6, 92.4, 128, 115, 212, 119, ... 152, 89.3, 164, 139, 57, 119, 115]
weight infor: [0.00187,

In [ ]:
PLACE_HOLDER

### some test about the output templates

In [56]:
with open(f"{plot_dir}/templates_2018.pkl","rb") as f:
    hists_template1 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value
# hists_template["CR2"]["QCD",:]
hists_template1["SR2a"]["data",:]

Hist(Regular(20, 50, 250, name='MH_Reco', label='Higgs candidate MET recovery mass [GeV]'), storage=Weight()) # Sum: WeightedSum(value=192, variance=192) (WeightedSum(value=207, variance=207) with flow)

In [60]:
hists_template1.keys()

dict_keys(['SR1a', 'SR1b', 'CR1', 'SR2a', 'SR2b', 'CR2', 'SR1aBlinded', 'SR1bBlinded', 'CR1Blinded', 'SR2aBlinded', 'SR2bBlinded', 'CR2Blinded'])

In [ ]:
hists_template1["SR1aBlinded"]

In [ ]:
sample_template1 = hists_template1["SR1a"]["QCD", :]
err = sample_template1.variances()
err

In [ ]:
for i , axis in enumerate(hists_template1["SR1a"].axes[1:]):
    print(i, axis)

### Some test of HHbbVV analysis for reference

In [57]:
# with open("/home/pku/zhaoyz/Higgs/HHbbVV/src/HHbbVV/postprocessing/templates/23Jun14/2018_templates.pkl","rb") as f:
with open("/ospool/cms-user/yuzhe/BoostedHWW/prediction/HHbbVV/src/HHbbVV/postprocessing/templates/24Mar15UpdateData/2018_templates.pkl","rb") as f:
    hists_template2 = pkl.load(f)
# hists_template["pass"]["QCD",:]  
# hists_template["pass"]["QCD",:].sum().value


In [58]:
hists_template2.keys()

dict_keys(['pass', 'fail', 'pass_JES_up', 'fail_JES_up', 'pass_JES_down', 'fail_JES_down', 'pass_JER_up', 'fail_JER_up', 'pass_JER_down', 'fail_JER_down', 'pass_JMS_up', 'fail_JMS_up', 'pass_JMS_down', 'fail_JMS_down', 'pass_JMR_up', 'fail_JMR_up', 'pass_JMR_down', 'fail_JMR_down', 'passBlinded', 'failBlinded', 'pass_JES_upBlinded', 'fail_JES_upBlinded', 'pass_JES_downBlinded', 'fail_JES_downBlinded', 'pass_JER_upBlinded', 'fail_JER_upBlinded', 'pass_JER_downBlinded', 'fail_JER_downBlinded', 'pass_JMS_upBlinded', 'fail_JMS_upBlinded', 'pass_JMS_downBlinded', 'fail_JMS_downBlinded', 'pass_JMR_upBlinded', 'fail_JMR_upBlinded', 'pass_JMR_downBlinded', 'fail_JMR_downBlinded'])

In [ ]:
hists_template2["pass"]

In [ ]:
hists_template2["pass"]["HHbbVV_pileup_up",:]

In [ ]:
hists_template2["pass"]["HHbbVV_pileup_up",:]

In [ ]:
hists_template2["pass"]["HHbbVV",:]

In [ ]:
hists_template2["passBlinded"]

### test how to load and use the *.pkl template file

In [ ]:
def get_template(h, sample):
    ''' 
    histogram h Hist, with axes:["samples","systematic","MH_Reco"]
    sample is sample name in ["QCD",...,"data"]
    '''
    mass_axis = 1 #axis index
    massbins = h.axes[mass_axis].edges
    return (h[sample, :].values(), massbins, "MH_Reco")

a = get_template(hists_template1["SR3a"],"QCD")
a

### Some other test

In [ ]:
regions = {
        "CR1" :{"SRa": "SR1a","SRb":"SR1b"},
        "CR2" :{"SRa": "SR2a","SRb":"SR2b"},
        "CR3" :{"SRa": "SR3a","SRb":"SR3b"},
        }

regions_blinded = { key_fail + "_blinded": {key_pass + "_blinded" : key_pass_ab + "_blinded" for key_pass , key_pass_ab in key_pass_dict.items()}  for key_fail , key_pass_dict in regions.items()}
regions_blinded.keys()

In [ ]:
region = "SR1a_blinded"
pass_region = ("a_" in region)
pass_region

In [ ]:
region = "SR1aBlinded"
region_noblinded = region.split("Blinded")[0]
region_noblinded

### function to scale the 